In [1]:
from tkinter import *
from PIL import Image, ImageTk
from tkinter import filedialog, Frame, ttk
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import glob

Cargamos el Dataset ya creado para entrenar un modelo

In [2]:
pickle_in = open('X.pickle','rb')
X = pickle.load(pickle_in)
pickle_in = open('y.pickle','rb')
y = pickle.load(pickle_in)
pickle_in = open('Z.pickle','rb')
Z = pickle.load(pickle_in)

In [3]:
# Divido entre datos de entrenamiento y de prueba
X_train, X_test, y_train, y_test, Z_train, Z_test = train_test_split(X, y, Z, stratify=y, random_state=1)

In [4]:
from sklearn.svm import SVC
svm = SVC(C=100)
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.64

Definimos funciones

In [5]:
#Dada una imágen predice el tipo de sello de la misma y lo devuelve como string
def predecir(imagen):
    img = cv2.imread(imagen, cv2.IMREAD_GRAYSCALE)
    img_N = cv2.resize(img,(300,300))
    img_V = (np.asarray(img_N)).flatten()
    #Dado un número nos dice cual es el tipo de sello al que está asociado ese número
    #Esto lo queremos pues knn nos devuelve números y nosotros los expresamos con el tipo de sello
    def clase(i):
        n=''
        if (i==0):
            n='Adenovirus'
        if (i==1):
            n='Ameba'
        if (i==2):
            n='Levadura'
        if (i==3):
            n='Penicilina'
        if (i==4):
            n='Phage'
        if (i==5):
            n='Sarcoptes'
        if (i==6):
            n='Sars-Cov-2'
        if (i==7):
            n='Sreptomyces coelicolor'
        if (i==8):
            n='Tardigrado'
        if (i==9):
            n='Tryponosoma'
        return n
    return(clase(svm.predict([img_V])))                

In [6]:
#Dada una imágen la recorta en imágenes más pequeñas y las guarda en la carpeta usuario
def recortar(imagen):
    for im in glob.glob("Usuario/*.jpg"):
        try:
            os.remove(im)
        except OSError as e:
            print(f"Error:{ e.strerror}")   
    image = imagen#cv2.imread(imagen,cv2.IMREAD_GRAYSCALE)
    gradX = cv2.Sobel(image, cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(image, cv2.CV_32F, dx=0, dy=1, ksize=-1)
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)
    blurred = cv2.blur(gradient, (9, 9))
    _, thresh = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    closed = cv2.erode(closed, None, iterations=4)
    closed = cv2.dilate(closed, None, iterations=4)
    a=100
    c=0
    dimensionesf = (closed.shape[0],closed.shape[1])
    while a>1:
        (cntsf, xf) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cf = sorted(cntsf, key=cv2.contourArea, reverse=True)[0]
        rectf = cv2.minAreaRect(cf)
        boxf = np.int0(cv2.boxPoints(rectf))
        Xsf = [i[0] for i in boxf]
        Ysf = [i[1] for i in boxf]
        x1f = min(Xsf)
        x2f = max(Xsf)
        y1f = min(Ysf)
        y2f = max(Ysf)
        cropImgf= image[y1f:y2f, x1f:x2f]
        cv2.imwrite("Usuario/imagen"+str(c)+".jpg", cropImgf)
        b=0
        arrayf=np.zeros(dimensionesf)
        for i in range(closed.shape[1]):
            for j in range(closed.shape[0]):
                if (x1f < i < x2f and y1f < j < y2f):
                    arrayf[j,i]=0
                else:
                    arrayf[j,i]=closed[j,i]
        for i in range(closed.shape[1]):
            for j in range(closed.shape[0]):
                if (arrayf[j,i]>0):
                    b=b+1
        a=(b*100)/(closed.shape[0]*closed.shape[1])
        c=c+1
        closed= np.uint8(arrayf)

In [7]:
def info(i):
    n='', ##Esta es la información
    if (i=='Adenovirus'):
        n='Los adenovirus (Adenoviridae)  Son virus no encapsulados de ADN bicatenario que pueden provocar enfermedades o malestares como infecciones en las vías respiratorias, conjuntivitis, cistitis hemorrágica y gastroenteritis. Estos infectan a humanos como a animales.'
    if (i=='Ameba'):
        n='Ameba, también conocida como Amoeba o Amiba. Es un protista unicelular del género Amoeba. Es un eucariota caracterizado por su forma cambiante, puesto que carece de pared celular, y por su movimiento ameboide a base de seudópodos, que también usa para capturar alimentos a través del proceso llamado fagocitosis.'
    if (i=='Levadura'):
        n='Se denomina levadura o fermento a cualquiera de los hongos microscópicos clasificados como ascomicetos o basidiomicetos, predominantemente unicelulares en su ciclo de vida, generalmente caracterizados por dividirse asexualmente por gemación o bipartición y por tener estados sexuales que no están adjuntos a un esporocarpo (cuerpo fructífero).'
    if (i=='Penicilina'):
        n='Las penicilinas son antibióticos empleados en el tratamiento de infecciones provocadas por bacterias sensibles; su descubrimiento ha sido atribuido a Alexander Fleming en 1928, quien obtuvo el Premio Nobel en Fisiología o Medicina en 1945 junto con los científicos Ernst Boris Chain y Howard Walter Florey, creadores de un método para producir el fármaco en masa.'
    if (i=='Phage'):
        n='Los bacteriófagos (también llamados fagos) son virus que infectan exclusivamente a las bacterias. Los fagos están constituidos por una cubierta proteica o cápside en cuyo interior está contenido su material genético, que puede ser ADN, de 5000 a 500 000 pares de bases.'
    if (i=='Sars-Cov-2'):
        n='Sus siglas en inglés significan “Severe acute respiratory syndrome coronavirus 2”, es un tipo de coronavirus causante de la enfermedad del COVID-19. Identificado por primera vez en Wuhan, China en 2019 y declarado un virus de emergencia internacional por la OMS en enero del 2020. '
    if (i=='Tardigrado'):
        n='Los tardígrados (Tardígrada), llamados comúnmente osos de agua. Pertenecen al filo de los ecdisozoos dentro del reino animal, caracterizado por ser invertebrados, protóstomos, segmentados y microscópicos.'
    if (i=='Sarcoptes'):
        n='También conocido como el “Arador de la sarna”, es una especie de ácaro que se hospeda en animales mamíferos produciendo la enfermedad conocida como sarna. Fuera del huésped suele vivir entre 2 y 4 días.'
    if (i=='Sreptomyces-Coelicolor'):
        n='Se encuentra entre los representantes mejor estudiados del género Streptomyces, éstos tienen un ciclo de vida de desarrollo notablemente complejo y gracias a ellos se producen aproximadamente la mitad de los antibióticos usados clínicamente y otros productos naturales de utilidad farmacéutica.'
    if (i=='Tryponosoma'):
        n='Microorganismo que pertenece al reino protista, es un parásito unicelular que infecta a distintos vertebrados, incluso humanos, causando las enfermedades de tripanosomiasis, por ejemplo, la enfermedad del sueño.'
    return n

Interfaz gráfica

In [8]:
def seleccionar_imagen():
    pass
    archivo = filedialog.askopenfilename(filetypes=(('Archivos de imagen', '*.png'),("jpeg files","*.jpg")))
    
    if archivo is not None:
        for widget in second_frame.winfo_children():
            widget.destroy()
            
        for widget in fimagen.winfo_children():
            widget.destroy()
            
        imagen = Image.open(archivo)

        imagen = imagen.resize((200, 200), Image.ANTIALIAS)

        imagen = ImageTk.PhotoImage(imagen)
            
            
        lbl_textimg=Label(fimagen,text='La imágen que seleccionaste:')
        lbl_textimg.pack(pady=10)
            
        lbl_imagen = Label(fimagen, image=imagen)
        lbl_imagen.image = imagen
        lbl_imagen.pack(pady=10)
            
    if archivo is not None:
        img = cv2.imread(archivo, cv2.IMREAD_GRAYSCALE)
        recortar(img)
        Numero=len(glob.glob("Usuario/*.jpg"))
        for i in range(Numero):
            exec('imagen_{} = Image.open("Usuario/imagen"+str(i)+".jpg")'.format(i))
            exec('imagen_{} = imagen_{}.resize((60, 60), Image.ANTIALIAS)'.format(i,i))
            exec('imagen_{} = ImageTk.PhotoImage(imagen_{})'.format(i,i))
            exec('lbl_imagen_{} = Label(second_frame, image=imagen_{})'.format(i,i))
            exec('lbl_imagen_{}.image = imagen_{}'.format(i,i))
            exec('lbl_imagen_{}.grid(row=i+3, column=0,pady=10, padx=10)'.format(i))
            exec('lbl_tipo_{}=Label(second_frame,text="La imágen es del tipo:")'.format(i))
            exec('lbl_tipo_{}.grid(row=i+3,column=1,sticky=E)'.format(i))
            exec('lbl_img_{}=Label(second_frame,text=predecir("Usuario/imagen"+str(i)+".jpg"),width=10)'.format(i))
            exec('lbl_img_{}.config(bg="white")'.format(i))
            exec('lbl_img_{}.grid(row=i+3,column=2,pady=15, padx=10,sticky=W)'.format(i))
            exec('lbl_inf_{}=Label(second_frame,text=info(predecir("Usuario/imagen"+str(i)+".jpg")))'.format(i))
            exec('lbl_inf_{}.grid(row=i+3,column=3,sticky="w")'.format(i))

root=Tk()
root.wm_title('Clasificador de imágenes')
root.geometry('1000x500')

fpedir=Frame(root, height=100, bg='cyan')
fpedir.pack(fill='x')
btn_seleccionar_imagen = Button(fpedir, text='Seleccionar imagen...', command=seleccionar_imagen)
btn_seleccionar_imagen.pack(pady=15)
        
fcontenido = Frame(root,bg='yellow')
fcontenido.pack(fill='both',expand=1)

fderecha=Frame(fcontenido)#,bg='blue')
fderecha.pack(side='right',fill='both',expand=1)
fimagen=Frame(fcontenido)#,bg='red')
fimagen.pack(side='left',fill='both',expand=1)
my_canvas=Canvas(fderecha)#,bg='green')
my_canvas.pack(side='left',fill='both',expand=1)

my_scrollbar=ttk.Scrollbar(fderecha,orient='vertical',command=my_canvas.yview)
my_scrollbar.pack(side='right',fill='y')

my_canvas.configure(yscrollcommand=my_scrollbar.set)
my_canvas.bind('<Configure>', lambda e: my_canvas.configure(scrollregion=my_canvas.bbox('all')))


my_scrollbar2=ttk.Scrollbar(fderecha,orient='horizontal',command=my_canvas.xview)
my_scrollbar2.pack(side='bottom',fill='x')
my_canvas.configure(xscrollcommand=my_scrollbar2.set)
my_canvas.bind('<Configure>', lambda e: my_canvas.configure(scrollregion=my_canvas.bbox('all')))

second_frame=Frame(my_canvas)#,bg='green')
second_frame.pack(side='right',fill='both',expand=1)

my_canvas.create_window((0,0),window=second_frame, anchor='nw')

root.mainloop()